In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import missingno as msno
from datetime import date
import researchpy as rp
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
from sklearn.preprocessing import MinMaxScaler
#pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [2]:
df = pd.read_excel("/Users/serhandulger/PycharmProjects/DSMLBC_7/WEEK_3/DATASETS/online_retail_II.xlsx", sheet_name="Year 2009-2010")

In [3]:
import datetime as dt
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], format="%Y-%m-%d %H:%M:%S")

In [4]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### NA SUM #####################")
    print(dataframe.isnull().sum().sum())
    print("##################### Describe #####################")
    print(dataframe.describe())
    print("##################### Nunique #####################")
    print(dataframe.nunique())

In [5]:
check_df(df)

##################### Shape #####################
(525461, 8)
##################### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
##################### Head #####################
  Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12 2009-12-01 07:45:00   6.95      13085.0  United Kingdom
1  489434    79323P                   PINK CHERRY LIGHTS        12 2009-12-01 07:45:00   6.75      13085.0  United Kingdom
2  489434    79323W                  WHITE CHERRY LIGHTS        12 2009-12-01 07:45:00   6.75      13085.0  United Kingdom
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48 2009-12-01 07:45:00   2.10

In [6]:
df[df["Invoice"].str.contains("C",na=False)].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia


In [7]:
def missing_values_analysis(df):
    na_columns_ = [col for col in df.columns if df[col].isnull().sum() > 0]
    n_miss = df[na_columns_].isnull().sum().sort_values(ascending=True)
    ratio_ = (df[na_columns_].isnull().sum() / df.shape[0] * 100).sort_values(ascending=True)
    missing_df = pd.concat([n_miss, np.round(ratio_, 2)], axis=1, keys=['Total Missing Values', 'Ratio'])
    missing_df = pd.DataFrame(missing_df)
    return missing_df

In [8]:
missing_values_analysis(df)

,Total Missing Values,Ratio
Description,2928,0.56
Customer ID,107927,20.54


In [9]:
df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[df["Quantity"]>0]
df.dropna(inplace=True)
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [10]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


In [11]:
missing_values_analysis(df)

,Total Missing Values,Ratio


In [13]:
cltv_calculation = df.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                        'Quantity': lambda x: x.sum(),
                                        'TotalPrice': lambda x: x.sum()})

In [15]:
cltv_calculation.head()

,Invoice,Quantity,TotalPrice
Customer ID,,,
12346.0,11,70,372.86
12347.0,2,828,1323.32
12348.0,1,373,222.16
12349.0,3,993,2671.14
12351.0,1,261,300.93


In [16]:
cltv_calculation.columns = ["total_transaction","total_unit","total_price"]

In [17]:
cltv_calculation.head()

,total_transaction,total_unit,total_price
Customer ID,,,
12346.0,11,70,372.86
12347.0,2,828,1323.32
12348.0,1,373,222.16
12349.0,3,993,2671.14
12351.0,1,261,300.93


In [18]:
##################################################
# 2. Average Order Value (average_order_value = total_price / total_transaction)
##################################################

cltv_calculation["avg_order_value"] = cltv_calculation["total_price"] / cltv_calculation["total_transaction"]

In [19]:
##################################################
# 3. Purchase Frequency (total_transaction / total_number_of_customers)
##################################################

cltv_calculation["purchase_frequency"] = cltv_calculation["total_transaction"] / cltv_calculation.shape[0]

In [27]:
##################################################
# 4. Repeat Rate & Churn Rate (birden fazla alışveriş yapan müşteri sayısı / tüm müşteriler)
##################################################

repeat_rate = cltv_calculation[cltv_calculation.total_transaction > 1].shape[0] / cltv_calculation.shape[0]
churn_rate = 1 - repeat_rate
print(f" The repeat rate for transaction ",repeat_rate)
print(f" The Churn rate is ",churn_rate)

 The repeat rate for transaction  0.6706073249884098
 The Churn rate is  0.3293926750115902


In [28]:
##################################################
# 5. Profit Margin (profit_margin =  total_price * 0.10)
##################################################

cltv_calculation["profit_margin"] = cltv_calculation["total_price"] * 0.10 

In [30]:
##################################################
# 6. Customer Value (customer_value = average_order_value * purchase_frequency)
##################################################

# Customer Value

cltv_calculation["customer_value"] = (cltv_calculation["avg_order_value"] * cltv_calculation["purchase_frequency"]) / churn_rate

In [32]:
##################################################
# 7. Customer Lifetime Value (CLTV = (customer_value / churn_rate) x profit_margin)
##################################################

cltv_calculation["cltv"] = cltv_calculation["customer_value"] * cltv_calculation["profit_margin"]

cltv_calculation.head()

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
12346.0,11,70,372.86,33.896364,0.002550,37.286,0.262393,9.783574
12347.0,2,828,1323.32,661.660000,0.000464,132.332,0.931260,123.235455
12348.0,1,373,222.16,222.160000,0.000232,22.216,0.156341,3.473263
12349.0,3,993,2671.14,890.380000,0.000695,267.114,1.879761,502.110408
12351.0,1,261,300.93,300.930000,0.000232,30.093,0.211773,6.372897


In [37]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(cltv_calculation[["cltv"]])
cltv_calculation["scaled_cltv"] = scaler.transform(cltv_calculation[["cltv"]])
cltv_calculation.sort_values(by="scaled_cltv", ascending=False).head()

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,customer_value,cltv,scaled_cltv,segment
Customer ID,,,,,,,,,,
18102.0,89,124216,349164.35,3923.194944,0.020631,34916.435,245.717347,8.579574e+06,1.000000,A
14646.0,78,170342,248396.50,3184.570513,0.018081,24839.650,174.804011,4.342070e+06,0.506094,A
14156.0,102,108107,196566.74,1927.124902,0.023644,19656.674,138.329866,2.719105e+06,0.316928,A
14911.0,205,69722,152147.57,742.183268,0.047520,15214.757,107.070774,1.629056e+06,0.189876,A
13694.0,94,125893,131443.19,1398.331809,0.021790,13144.319,92.500486,1.215856e+06,0.141715,A


In [46]:
##################################################
# 8. Creating Segments based on CLTV Value of Customers
##################################################

cltv_calculation["segment"] = pd.qcut(cltv_calculation["scaled_cltv"], 4, labels=["D", "C", "B", "A"])
cltv_calculation.head()

,total_transaction,total_unit,total_price,avg_order_value,purchase_frequency,profit_margin,customer_value,cltv,scaled_cltv,segment
Customer ID,,,,,,,,,,
12346.0,11,70,372.86,33.896364,0.002550,37.286,0.262393,9.783574,1.140333e-06,C
12347.0,2,828,1323.32,661.660000,0.000464,132.332,0.931260,123.235455,1.436382e-05,B
12348.0,1,373,222.16,222.160000,0.000232,22.216,0.156341,3.473263,4.048293e-07,D
12349.0,3,993,2671.14,890.380000,0.000695,267.114,1.879761,502.110408,5.852393e-05,A
12351.0,1,261,300.93,300.930000,0.000232,30.093,0.211773,6.372897,7.427988e-07,D


In [42]:
cltv_calculation[["total_transaction", "total_unit", "total_price", "cltv", "scaled_cltv","segment"]].sort_values(by="scaled_cltv",
                                                                                              ascending=False).head()

,total_transaction,total_unit,total_price,cltv,scaled_cltv,segment
Customer ID,,,,,,
18102.0,89,124216,349164.35,8.579574e+06,1.000000,A
14646.0,78,170342,248396.50,4.342070e+06,0.506094,A
14156.0,102,108107,196566.74,2.719105e+06,0.316928,A
14911.0,205,69722,152147.57,1.629056e+06,0.189876,A
13694.0,94,125893,131443.19,1.215856e+06,0.141715,A


#### Different Approach to Calculate CLV

In [49]:
def get_month(x):
    return dt.datetime(x.year,x.month,1)

In [50]:
def get_date(df,column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year,month,day

In [53]:
def calculate_cohort(df):
    df["InvoiceMonth"] = df["InvoiceDate"].apply(get_month)
    
    grouping = df.groupby(["Customer ID"])["InvoiceMonth"]
    
    df["CohortMonth"] = grouping.transform("min")

    invoice_year, invoice_month, _ = get_date(df,"InvoiceMonth")

    cohort_year, cohort_month, _ = get_date(df,"CohortMonth")

    # Calculate difference in years

    years_diff = invoice_year - cohort_year

    # Calculate difference in months

    months_diff = invoice_month - cohort_month

    df["CohortIndex"] = years_diff*12+months_diff+ 1
    cohort_data = df.groupby(["CohortMonth","CohortIndex"])["Customer ID"].nunique()
    cohort_data = cohort_data.reset_index()
    cohort_counts = cohort_data.pivot(index="CohortMonth",
                                 columns="CohortIndex",
                                 values="Customer ID")
    
    cohort_sizes = cohort_counts.iloc[:,0]
    retention = cohort_counts.divide(cohort_sizes,axis=0)
    retention_rate = retention.round(3)*100
    return cohort_sizes,retention,retention_rate

In [55]:
cohort_sizes,retention,retention_rate = calculate_cohort(df)

In [56]:
retention

CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2009-12-01,1.0,0.352880,0.334031,0.425131,0.380105,0.359162,0.376963,0.342408,0.336126,0.362304,0.421990,0.495288,0.248168
2010-01-01,1.0,0.206266,0.310705,0.305483,0.263708,0.300261,0.258486,0.229765,0.279373,0.318538,0.302872,0.099217,NaN
2010-02-01,1.0,0.236702,0.223404,0.289894,0.244681,0.199468,0.191489,0.284574,0.252660,0.273936,0.071809,NaN,NaN
2010-03-01,1.0,0.189616,0.230248,0.241535,0.232506,0.203160,0.246050,0.302483,0.275395,0.079007,NaN,NaN,NaN
2010-04-01,1.0,0.193878,0.193878,0.163265,0.183673,0.224490,0.275510,0.261905,0.068027,NaN,NaN,NaN,NaN
2010-05-01,1.0,0.157480,0.169291,0.173228,0.177165,0.255906,0.212598,0.078740,NaN,NaN,NaN,NaN,NaN
2010-06-01,1.0,0.174074,0.188889,0.203704,0.229630,0.285185,0.066667,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-01,1.0,0.155914,0.182796,0.295699,0.290323,0.102151,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-01,1.0,0.203704,0.296296,0.320988,0.117284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
churn = 1 - retention
churn

CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2009-12-01,0.0,0.647120,0.665969,0.574869,0.619895,0.640838,0.623037,0.657592,0.663874,0.637696,0.578010,0.504712,0.751832
2010-01-01,0.0,0.793734,0.689295,0.694517,0.736292,0.699739,0.741514,0.770235,0.720627,0.681462,0.697128,0.900783,NaN
2010-02-01,0.0,0.763298,0.776596,0.710106,0.755319,0.800532,0.808511,0.715426,0.747340,0.726064,0.928191,NaN,NaN
2010-03-01,0.0,0.810384,0.769752,0.758465,0.767494,0.796840,0.753950,0.697517,0.724605,0.920993,NaN,NaN,NaN
2010-04-01,0.0,0.806122,0.806122,0.836735,0.816327,0.775510,0.724490,0.738095,0.931973,NaN,NaN,NaN,NaN
2010-05-01,0.0,0.842520,0.830709,0.826772,0.822835,0.744094,0.787402,0.921260,NaN,NaN,NaN,NaN,NaN
2010-06-01,0.0,0.825926,0.811111,0.796296,0.770370,0.714815,0.933333,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-01,0.0,0.844086,0.817204,0.704301,0.709677,0.897849,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-01,0.0,0.796296,0.703704,0.679012,0.882716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
# Calculating the mean of retention rate
retention_rate = retention.iloc[:,1:].mean().mean()

# Calculating the mean of churn rate

churn_rate = churn.iloc[:,1:].mean().mean()

# Print rounded retention and churn rates
print('Retention rate: {:.2f}; Churn rate: {:.2f}'.format(retention_rate, churn_rate))

Retention rate: 0.25; Churn rate: 0.75


In [65]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice,customer_value,InvoiceMonth,CohortMonth,CohortIndex
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4,NaN,2009-12-01,2009-12-01,1
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,NaN,2009-12-01,2009-12-01,1
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,NaN,2009-12-01,2009-12-01,1
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8,NaN,2009-12-01,2009-12-01,1
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0,NaN,2009-12-01,2009-12-01,1


In [73]:
##################################################
# CALCULATING BASIC CLV

# Average Monthly Spent * Projected Customer Lifespan
##################################################

# Calculating monthly spending per customer
monthly_revenue = df.groupby(["Customer ID","InvoiceMonth"])["TotalPrice"].sum()

# Calculating average monthly spend
monthly_revenue = np.mean(monthly_revenue)
monthly_revenue

# Define lifespan to 36 months
lifespan_months = 36

# Calculate basic CLV
clv_basic = monthly_revenue * lifespan_months

# Print basic CLV value
print('Average basic CLV is {:.1f} USD'.format(clv_basic))

Average basic CLV is 24241.5 USD


In [76]:
##################################################
# CALCULATING GRANULAR CLV

# It will focus on more granular data points at the invoice level. 
##################################################

# Calculating average revenue per invoice
revenue_per_purchase = df.groupby(['Invoice'])['TotalPrice'].mean().mean()

# Calculating average number of unique invoices per customer per month
frequency_per_month = df.groupby(['Customer ID','InvoiceMonth'])['Invoice'].nunique().mean()

# Define lifespan to 36 months
lifespan_months = 36

# Calculating granular CLV
clv_granular = revenue_per_purchase * frequency_per_month * lifespan_months

# All together
print('Average granular CLV is {:.1f} USD'.format(clv_granular))
print('Revenue Per Purchase {:.1f} USD'.format(revenue_per_purchase))
print('Frequency Per Month {:.1f} USD'.format(frequency_per_month))

Average granular CLV is 2795.6 USD
Revenue Per Purchase 53.0 USD
Frequency Per Month 1.5 USD


In [77]:
##################################################
# CALCULATING TRADITIONAL CLV
##################################################

# Calculate monthly spend per customer
monthly_revenue = df.groupby(['Customer ID','InvoiceMonth'])['TotalPrice'].sum().mean()

# Calculate average monthly retention rate
retention_rate = retention.iloc[:,1:].mean().mean()

# Calculate average monthly churn rate
churn_rate = 1 - retention_rate

# Calculate traditional CLV 
clv_traditional = monthly_revenue * (retention_rate / churn_rate)

# Print traditional CLV and the retention rate values
print('Average traditional CLV is {:.1f} USD at {:.1f} % retention_rate'.format(clv_traditional, retention_rate*100))

Average traditional CLV is 219.6 USD at 24.6 % retention_rate
